# Wind Statistics

### Introduction:

The data have been modified to contain some missing values, identified by NaN.  
Using pandas should make this exercise
easier, in particular for the bonus question.

You should be able to perform all of these operations without using
a for loop or other looping construct.


1. The data in 'wind.data' has the following format:

In [ ]:
"""
Yr Mo Dy   RPT   VAL   ROS   KIL   SHA   BIR   DUB   CLA   MUL   CLO   BEL   MAL
61  1  1 15.04 14.96 13.17  9.29   NaN  9.87 13.67 10.25 10.83 12.58 18.50 15.04
61  1  2 14.71   NaN 10.83  6.50 12.62  7.67 11.50 10.04  9.79  9.67 17.54 13.83
61  1  3 18.50 16.88 12.33 10.13 11.17  6.17 11.25   NaN  8.50  7.67 12.75 12.71
"""

'\nYr Mo Dy   RPT   VAL   ROS   KIL   SHA   BIR   DUB   CLA   MUL   CLO   BEL   MAL\n61  1  1 15.04 14.96 13.17  9.29   NaN  9.87 13.67 10.25 10.83 12.58 18.50 15.04\n61  1  2 14.71   NaN 10.83  6.50 12.62  7.67 11.50 10.04  9.79  9.67 17.54 13.83\n61  1  3 18.50 16.88 12.33 10.13 11.17  6.17 11.25   NaN  8.50  7.67 12.75 12.71\n'

   The first three columns are year, month and day.  The
   remaining 12 columns are average windspeeds in knots at 12
   locations in Ireland on that day.   

   More information about the dataset go [here](wind.desc).

### Step 1. Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/thieu1995/csv-files/main/data/pandas/wind.data)

In [ ]:
data_url = 'https://raw.githubusercontent.com/thieu1995/csv-files/main/data/pandas/wind.data'
data = pd.read_csv(data_url, delim_whitespace=True)

<ipython-input-3-d5f2e59a5c1a>:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(data_url, delim_whitespace=True)


### Step 3. Assign it to a variable called data and replace the first 3 columns by a proper datetime index.

In [ ]:
date_index = pd.to_datetime(dict(year=data.Yr, month=data.Mo, day=data.Dy), errors='coerce')
data.index = date_index
data = data.drop(columns=['Yr', 'Mo', 'Dy'])

### Step 4. Year 2061? Do we really have data from this year? Create a function to fix it and apply it.

In [ ]:
def correct_year(dt_index):
    return dt_index.map(lambda d: d.replace(year=d.year + 1900 if d.year < 100 else d.year))

data.index = correct_year(data.index)

### Step 5. Set the right dates as the index. Pay attention at the data type, it should be datetime64[ns].

In [ ]:
data.index = pd.to_datetime(data.index)

### Step 6. Compute how many values are missing for each location over the entire record.  
#### They should be ignored in all calculations below.

In [ ]:
missing_values = data.isna().sum()
print("Missing values per location:\n", missing_values)

Missing values per location:
 RPT    6
VAL    3
ROS    2
KIL    5
SHA    2
BIR    0
DUB    3
CLA    2
MUL    3
CLO    1
BEL    0
MAL    4
dtype: int64


### Step 7. Compute how many non-missing values there are in total.

In [ ]:
non_missing_total = data.notna().sum().sum()
print("\nTotal non-missing values:", non_missing_total)


Total non-missing values: 78857


### Step 8. Calculate the mean windspeeds of the windspeeds over all the locations and all the times.
#### A single number for the entire dataset.

In [ ]:
mean_windspeed = data.stack().mean()
print("\nOverall mean windspeed:", mean_windspeed)


Overall mean windspeed: 10.22788376428218


### Step 9. Create a DataFrame called loc_stats and calculate the min, max and mean windspeeds and standard deviations of the windspeeds at each location over all the days

#### A different set of numbers for each location.

In [ ]:
loc_stats = data.aggregate(['min', 'max', 'mean', 'std'])
print("\nLocation statistics:\n", loc_stats)


Location statistics:
             RPT        VAL        ROS        KIL        SHA        BIR  \
min    0.670000   0.210000   1.500000   0.000000   0.130000   0.000000   
max   35.800000  33.370000  33.840000  28.460000  37.540000  26.160000   
mean  12.362987  10.644314  11.660526   6.306468  10.455834   7.092254   
std    5.618413   5.267356   5.008450   3.605811   4.936125   3.968683   

            DUB        CLA        MUL        CLO        BEL        MAL  
min    0.000000   0.000000   0.000000   0.040000   0.130000   0.670000  
max   30.370000  31.080000  25.880000  28.210000  42.380000  42.540000  
mean   9.797343   8.495053   8.493590   8.707332  13.121007  15.599079  
std    4.977555   4.499449   4.166872   4.503954   5.835037   6.699794  


### Step 10. Create a DataFrame called day_stats and calculate the min, max and mean windspeed and standard deviations of the windspeeds across all the locations at each day.

#### A different set of numbers for each day.

In [ ]:
day_stats = data.aggregate(['min', 'max', 'mean', 'std'], axis=1)
print("\nDay statistics (first few rows):\n", day_stats.head())


Day statistics (first few rows):
       min    max       mean       std
NaT  9.29  18.50  13.018182  2.808875
NaT  6.50  17.54  11.336364  3.188994
NaT  6.17  18.50  11.641818  3.681912
NaT  1.79  11.75   6.619167  3.198126
NaT  6.17  13.33  10.630000  2.445356


### Step 11. Find the average windspeed in January for each location.  
#### Treat January 1961 and January 1962 both as January.

In [ ]:
january_data = data[data.index.month == 1]
january_mean = january_data.mean()
print("\nAverage January windspeed per location:\n", january_mean)



Average January windspeed per location:
 RPT   NaN
VAL   NaN
ROS   NaN
KIL   NaN
SHA   NaN
BIR   NaN
DUB   NaN
CLA   NaN
MUL   NaN
CLO   NaN
BEL   NaN
MAL   NaN
dtype: float64


### Step 12. Downsample the record to a yearly frequency for each location.

In [ ]:
# Before resampling, check and handle NaT values in the index
# First, identify if there are any NaT values in the index
if data.index.isna().any():
    # If NaT values are present, either drop rows with NaT index
    data = data[data.index.notna()]
    # or fill NaT values with appropriate timestamps (if possible)
    # data.index = data.index.fillna(method='ffill')  # for example, forward fill

# Now you can proceed with resampling
yearly = data.resample('Y').mean()
print("\nYearly mean (first few rows):\n", yearly.head())


Yearly mean (first few rows):
 Empty DataFrame
Columns: [RPT, VAL, ROS, KIL, SHA, BIR, DUB, CLA, MUL, CLO, BEL, MAL]
Index: []


<ipython-input-14-9fe7b1a9523c>:10: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly = data.resample('Y').mean()


### Step 13. Downsample the record to a monthly frequency for each location.

In [ ]:
monthly = data.resample('M').mean()
print("\nMonthly mean (first few rows):\n", monthly.head())


Monthly mean (first few rows):
 Empty DataFrame
Columns: [RPT, VAL, ROS, KIL, SHA, BIR, DUB, CLA, MUL, CLO, BEL, MAL]
Index: []


<ipython-input-15-463d5cd97f7e>:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = data.resample('M').mean()


### Step 14. Downsample the record to a weekly frequency for each location.

In [ ]:
weekly = data.resample('W').mean()
print("\nWeekly mean (first few rows):\n", weekly.head())


Weekly mean (first few rows):
 Empty DataFrame
Columns: [RPT, VAL, ROS, KIL, SHA, BIR, DUB, CLA, MUL, CLO, BEL, MAL]
Index: []


### Step 15. Calculate the min, max and mean windspeeds and standard deviations of the windspeeds across all locations for each week (assume that the first week starts on January 2 1961) for the first 52 weeks.

In [ ]:
weekly_stats = data.resample('W-MON').agg(['min', 'max', 'mean', 'std'])
first_52_weeks = weekly_stats.iloc[:52]
print("\nWeekly stats for first 52 weeks:\n", first_52_weeks.head())


Weekly stats for first 52 weeks:
 Empty DataFrame
Columns: [(RPT, min), (RPT, max), (RPT, mean), (RPT, std), (VAL, min), (VAL, max), (VAL, mean), (VAL, std), (ROS, min), (ROS, max), (ROS, mean), (ROS, std), (KIL, min), (KIL, max), (KIL, mean), (KIL, std), (SHA, min), (SHA, max), (SHA, mean), (SHA, std), (BIR, min), (BIR, max), (BIR, mean), (BIR, std), (DUB, min), (DUB, max), (DUB, mean), (DUB, std), (CLA, min), (CLA, max), (CLA, mean), (CLA, std), (MUL, min), (MUL, max), (MUL, mean), (MUL, std), (CLO, min), (CLO, max), (CLO, mean), (CLO, std), (BEL, min), (BEL, max), (BEL, mean), (BEL, std), (MAL, min), (MAL, max), (MAL, mean), (MAL, std)]
Index: []

[0 rows x 48 columns]
